<h2>Bubble Sheet Scanner</h2>
<p>Making a bubble sheet includes 7 steps.</p>
<font color = "#75776">
<h5>1.Detect the test sheet. <br>
    2.Apply Persective Transform to extract the top-down, birds-eye view. <br>
    3.Extract the set of bubbles. <br>
    4.Sort the bubbles into rows. <br>
    5.Determine the marked bubbles in each row. <br>
    6.Look for the correct answer. <br>
    7.Repeat for all questions. <br>
</h5>
    </font>

In [2]:
#importing required libraries
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

In [125]:
#constructing the argument parser
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True,
    #help = "Path to the image to be scanned")
args = vars(ap.parse_args())

SyntaxError: unexpected EOF while parsing (<ipython-input-125-0eb48f80c4e8>, line 5)

In [3]:
#defining answer key
ANSWER_KEY = {0:1,1:4,2:0,3:3,4:1}

In [13]:
#loading image and converting it to greyscale, applying gaussian blur and edging
image = cv2.imread("test_05.png") #change this to image["args"] in order to take argument from the user in the terminal
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

In [5]:
#finding contours
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

if len(cnts) > 0:
	# sort the contours according to their size in
	# descending order
	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

	# loop over the sorted contours
	for c in cnts:
		# approximate the contour
		peri = cv2.arcLength(c, True)
		approx = cv2.approxPolyDP(c, 0.02 * peri, True)

		# if our approximated contour has four points,
		# then we can assume we have found the paper
		if len(approx) == 4:
			docCnt = approx
			break


<h2>Four Point Transform</h2>
<p>Working:</p>
<ol>
<li>Orders the (x,y) coordinateds of our contours in a specific, reproducible manner</li>
<li>Applies perspective transform</li>
</ol>
<p>Read more at:<a> https://www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/ </a> </p>

In [6]:
#applying a four point perspective transform to both origional and grayscaled image
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

In [7]:
#Now for the marking, we need binarization, which means threshing of foreground from the background
#applying Otsu's thresholding method to binarize the warped image
thresh = cv2.threshold(warped, 0, 255,
	cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [8]:
# find contours in the thresholded image, then initialize
# the list of contours that correspond to questions
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []
 
# loop over the contours
for c in cnts:
	# compute the bounding box of the contour, then use the
	# bounding box to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(c)
	ar = w / float(h)

	# in order to label the contour as a question, region
	# should be sufficiently wide, sufficiently tall, and
	# have an aspect ratio approximately equal to 1
	if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
		questionCnts.append(c)


In [10]:
# sort the question contours top-to-bottom, then initialize
# the total number of correct answers
questionCnts = contours.sort_contours(questionCnts,
	method="top-to-bottom")[0]
correct = 0
 
# each question has 5 possible answers, to loop over the
# question in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
	# sort the contours for the current question from
	# left to right, then initialize the index of the
	# bubbled answer
	cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
	bubbled = None

	# loop over the sorted contours
	for (j, c) in enumerate(cnts):
		# construct a mask that reveals only the current
		# "bubble" for the question
		mask = np.zeros(thresh.shape, dtype="uint8")
		cv2.drawContours(mask, [c], -1, 255, -1)

		# apply the mask to the thresholded image, then
		# count the number of non-zero pixels in the
		# bubble area
		mask = cv2.bitwise_and(thresh, thresh, mask=mask)
		total = cv2.countNonZero(mask)

		# if the current total has a larger number of total
		# non-zero pixels, then we are examining the currently
		# bubbled-in answer
		if bubbled is None or total > bubbled[0]:
			bubbled = (total, j)

	# initialize the contour color and the index of the
	# *correct* answer
	color = (0, 0, 255)
	k = ANSWER_KEY[q]

	# check to see if the bubbled answer is correct
	if k == bubbled[1]:
		color = (0, 255, 0)
		correct += 1

	# draw the outline of the correct answer on the test
	cv2.drawContours(paper, [cnts[k]], -1, color, 3)


In [12]:
# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] score: 100.00%
